In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#this is sample data
from nltk.corpus import names  

from string import punctuation

In [2]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('names')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MR_SA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MR_SA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\MR_SA\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MR_SA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\MR_SA\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
#initilize function to do sentiment analysis
sid = SentimentIntensityAnalyzer()

In [4]:
eng_stopwords = stopwords.words('english')
eng_stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [5]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('countiesinfo1.csv')
df.head()


,County,FIPS code,County seat,Established,Origin,Meaning of name,Population,Area
0,Adams County\n,1,Gettysburg,1800,Parts of York County.,"John Adams,second U.S. President\n","7005101407000000000♠101,407\n",7002522000000000000♠522 sq mi(7003135200000000...
1,Allegheny County\n,3,Pittsburgh,1788,Parts of Washington and Westmoreland Counties.,"Lenape word for the Allegheny River, which pos...","7006122334800000000♠1,223,348\n",7002745000000000000♠745 sq mi(7003193000000000...
2,Armstrong County\n,5,Kittanning,1800,"Parts of Allegheny, Lycoming, and Westmoreland...","John Armstrong, Revolutionary War general\n","7004689410000000000♠68,941\n",7002664000000000000♠664 sq mi(7003172000000000...
3,Beaver County\n,7,Beaver,1800,Parts of Allegheny and Washington Counties.,"Beaver River, itself named for the eponymous a...","7005170539000000000♠170,539\n",7002444000000000000♠444 sq mi(7003115000000000...
4,Bedford County\n,9,Bedford,1771,Parts of Cumberland County.,"Fort Bedford, which is named for John Russell,...","7004497620000000000♠49,762\n","7003101500000000000♠1,015 sq mi(70032629000000..."


In [7]:
df=df.drop(['County','FIPS code','Established','Origin','Population','Area'],axis=1)

In [8]:
df.head()

,County seat,Meaning of name
0,Gettysburg,"John Adams,second U.S. President\n"
1,Pittsburgh,"Lenape word for the Allegheny River, which pos..."
2,Kittanning,"John Armstrong, Revolutionary War general\n"
3,Beaver,"Beaver River, itself named for the eponymous a..."
4,Bedford,"Fort Bedford, which is named for John Russell,..."


In [9]:
df=df.drop(['County seat'],axis=1)

In [10]:
df.head()

,Meaning of name
0,"John Adams,second U.S. President\n"
1,"Lenape word for the Allegheny River, which pos..."
2,"John Armstrong, Revolutionary War general\n"
3,"Beaver River, itself named for the eponymous a..."
4,"Fort Bedford, which is named for John Russell,..."


In [11]:
# rename column headers
colheader = {'Meaning of name': 'Review Text'
             
             
    }

df.rename(columns=colheader, inplace=True)

In [12]:
df.head()

,Review Text
0,"John Adams,second U.S. President\n"
1,"Lenape word for the Allegheny River, which pos..."
2,"John Armstrong, Revolutionary War general\n"
3,"Beaver River, itself named for the eponymous a..."
4,"Fort Bedford, which is named for John Russell,..."


In [14]:
df.isnull().sum()

Review Text    0
dtype: int64

In [15]:
#create a function to clean up each review
#then it will analyze and assign a sentiment polarity
def reviewSentiment(review):
    
    #make text lowercase
    review = review.lower()
    
    #tokenize the review
    #tknz_review is a list
    tknz_review = word_tokenize(review)
    
    #remove puntuation
    for token in tknz_review:
        if token in punctuation:
            tknz_review.remove(token)
    
    #empty list to hold "cleaned" tokens
    clean_tokens = []
    
    #remove filler words
    for token in tknz_review:
        if token not in eng_stopwords:
            clean_tokens.append(token)
            
    #put sentence back together with remaining clean words
    clean_review = ' '.join(clean_tokens)
    
    #get the polarity scores dictionary
    sid_rev = sid.polarity_scores(clean_review)
    
    #get sentiment polarity from the "compound" key in the sid_rev dictionary
    r_comp = sid_rev['compound']
    
    #return the sentiment value
    return r_comp

In [16]:
#create a new column to hold sentiment value from function
df['review_sentiment'] = df['Review Text'].apply(reviewSentiment)

In [17]:
df.head()

,Review Text,review_sentiment
0,"John Adams,second U.S. President\n",0.0000
1,"Lenape word for the Allegheny River, which pos...",0.5994
2,"John Armstrong, Revolutionary War general\n",-0.5994
3,"Beaver River, itself named for the eponymous a...",0.0000
4,"Fort Bedford, which is named for John Russell,...",0.0000


In [18]:
df.dtypes

Review Text          object
review_sentiment    float64
dtype: object

In [19]:
#create a function to assign a polarity category to the sentiment
def sentimentCategory(sent_num):
    if sent_num >= 0.2:
        return "positive"
    if sent_num <= -0.2:
        return "negative"
    else:
        return "neutral"

In [20]:
#create a new column to hold sentiment category
df['sentiment_category'] = df['review_sentiment'].apply(sentimentCategory)

In [21]:
df.head()

,Review Text,review_sentiment,sentiment_category
0,"John Adams,second U.S. President\n",0.0000,neutral
1,"Lenape word for the Allegheny River, which pos...",0.5994,positive
2,"John Armstrong, Revolutionary War general\n",-0.5994,negative
3,"Beaver River, itself named for the eponymous a...",0.0000,neutral
4,"Fort Bedford, which is named for John Russell,...",0.0000,neutral


In [22]:
#compare frequency of positive, negative, and neutral reviews
df['sentiment_category'].value_counts()

neutral     47
negative    12
positive     8
Name: sentiment_category, dtype: int64

In [23]:
df['Review Text'].iloc[0]

'John Adams,second U.S. President\n'

In [24]:
df['review_sentiment'].iloc[0]

0.0

In [25]:
df['sentiment_category'].iloc[0]

'neutral'

In [26]:
df['Review Text'].iloc[4]

'Fort Bedford, which is named for John Russell, 4th Duke of Bedford\n'

In [27]:
df['review_sentiment'].iloc[4]

0.0

In [28]:
df['sentiment_category'].iloc[4]

'neutral'